# Fully Connected Deep Neural Network
## 预处理

In [1]:
from others import load_all_dataset, rename_dataset
X_train, y_train, X_test, y_test = load_all_dataset(show=False)
import numpy as np
np.set_printoptions(edgeitems=5,
                    linewidth=1000,
                    formatter={"float":lambda x: "{:.3f}".format(x)})

Train data
Test data


In [3]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
import warnings
warnings.filterwarnings("ignore")

# 去除NaN
class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        ''' Replace NaN by 0 and flatten the matrix to size (sample, 6720).
        Executed on every input data (i.e., source, bkg, target) and passed
        the resulting arrays to `fit`and `predict` methods in :class: Classifier

        Parameters
        ----------
        `X`: ndarray of (sample, 672, 10)
            3D input dataset(sample, time, features)
        
        Returns
        -------
        `X`: ndarray of (sample, 6720)
            The filtered dataset
        '''
        #! ATTENTION
        # The idea is supposed to eliminate the common columns filled entirely 
        # by NaN. But in this competition, since we don't have access to
        # `OpticalDataset` object, it's impossible to communicate informations
        # between datasets. So, here it deletes columns that are found on public
        # dataset.
        X = np.delete(X, [3,], axis=2)
        X = X.astype(np.float64)
        
        ## 1st round
        X1, nanmean = [], []
        for i in range(X.shape[0]):
            x = X[i]
            indice = ~np.isfinite(x)
            nanmean.append(np.nanmean(x, axis=0))

            # Columns with full Nan
            col_is_nan = np.all(indice, axis=0)
            if (col_is_nan == True).any():
                X1.append(x) # deal later
                continue
            
            # Rows with full Nan
            # Unachievable. Cause we don't have access to manipulate on labels
            # row_is_nan = np.all(indice, axis=1)
            # if (row_is_nan == True).any():
            #     row = np.where(row_is_nan == True)[0]
            #     if len(row) >= x.shape[0]/4: # drop sample, /2=85%+, /4=75%+
            #         continue
            
            # Columns with partial NaN
            part_is_nan = np.any(indice, axis=0)
            if (part_is_nan == True).any():
                col = np.where(part_is_nan == True)[0]
                # part_nan[i] = col[0]
                for c in col:
                    this = x[:,c]
                    finite = this[np.isfinite(this)]
                    fill = np.repeat(finite, np.ceil(len(this)/len(finite)))[:len(this)]
                    x[:,c] = np.where(np.isfinite(this), this, fill)
            
            # Construct new array
            X1.append(x)
        X1, nanmean = np.array(X1), np.array(nanmean)

        ## 2nd round
        candidate_mean = []
        for i in range(nanmean.shape[1]):
            col = nanmean[i]
            finite = col[np.isfinite(col)]
            candidate_mean.append(finite)

        X2 = []
        for i in range(X1.shape[0]):
            x = X[i]
            indice = ~np.isfinite(x)
            # Columns with full Nan
            col_is_nan = np.all(indice, axis=0)
            if (col_is_nan == True).any():
                col = np.where(col_is_nan == True)[0]
                for c in col:
                    value = np.random.choice(candidate_mean[c])
                    x = np.nan_to_num(x, nan=value)
            X2.append(x)
        
        X = np.array(X2)

        ## Final
        X = X.reshape(X.shape[0], -1) # Flatten
        # print("Expected True:", np.all(np.isfinite(X))) # expected True
        return X

fe = FeatureExtractor()

[X_source, X_source_bkg, X_target, X_target_unlabeled, X_target_bkg,
    y_source, y_target, X_test] = rename_dataset(
    fe, X_train, y_train, X_test, y_test, show_imbalance=0)

==== TRAIN SET ====
Expected True: True
  | X_source: (46110, 6048) ; y_source: (46110,)


<ipython-input-2-cd0deba31c2a>:36: RuntimeWarning: Mean of empty slice
  nanmean.append(np.nanmean(x, axis=0))


Expected True: True
A | X_source_bkg: (50862, 6048)
Expected True: True
----
  | X_target: (438, 6048) ; y_target: (438,)
Expected True: True
B | X_target_bkg: (29592, 6048)
Expected True: True
  | X_target_unlabeled: (8202, 6048)
==== TEST SET ====
Expected True: True
  | X_test.target: (17758, 6048) ; y_test.target: (17758,)
Expected True: True
B | X_test.target_bkg: (47275, 6048)
  | X_test.target_unlabeled: None


笔记:
- batch size 越大，同样多epoch下，acc 越小

In [4]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_source, y_source)).batch(32)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_target, y_target)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
# train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) ) # array must be 3D
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

In [10]:
train_dataset

<RepeatDataset shapes: ((None, 6048), (None,)), types: (tf.float64, tf.float32)>

## 搭建网络模型
参考资料

1. [Build your first Neural Network in TensorFlow 2](https://towardsdatascience.com/building-your-first-neural-network-in-tensorflow-2-tensorflow-for-hackers-part-i-e1e2f1dfe7a0)

In [5]:
# Sequential groups a linear stack of layers into a tf.keras.Model.
# https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
model = tf.keras.Sequential()
model.add( layers.Flatten(input_shape=(6048,), name="Input_Layer") )

num_fully_connected_layers = 10
for i in range(num_fully_connected_layers):
    model.add( layers.Dense(256, activation="relu", name="Layer{}".format(i+1)) )

model.add( layers.Dropout(0.5, name="Layer-1") )
model.add( layers.Dense(1, activation='sigmoid', name="Output_Layer") )

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(),
                       # tf.keras.metrics.PrecisionAtRecall(recall=0.1),
                       "acc",
                      ]
             )
# model.summary()

## 训练模型
- if 'softmax' in the last layer, output is 0 or 1
    - 'categorical_crossentropy' returns NaN, 'binary_crossentropy' acc ~ 0.1
- if `'sigmoid'` in the last layer, output is the probability of 1
    - 'categorical_crossentropy' returns NaN, `'binary_crossentropy'` acc ~ 0.8 to 0.9

In [6]:
import datetime as dt
callbacks = [
    # Write TensorBoard logs to `./logs` directory
    tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
        dt.datetime.now().strftime("%Y-%m-%d-%H-%M DNN")), write_images=True),
    ]
model.fit(train_dataset, epochs=100, steps_per_epoch=200,
          validation_data=valid_dataset, validation_steps=3,
          # callbacks=callbacks
          )

Epoch 1/100
200/200 [==============================] - 4s 12ms/step - loss: 61574.6055 - precision: 0.1607 - acc: 0.7755 - val_loss: 2696.6340 - val_precision: 0.5000 - val_acc: 0.6979
Epoch 2/100
200/200 [==============================] - 2s 10ms/step - loss: 10607.0527 - precision: 0.1201 - acc: 0.8031 - val_loss: 28215.2520 - val_precision: 0.0000e+00 - val_acc: 0.6979
Epoch 3/100
200/200 [==============================] - 2s 9ms/step - loss: 13843.8096 - precision: 0.1345 - acc: 0.8019 - val_loss: 1946.8374 - val_precision: 0.4688 - val_acc: 0.6771
Epoch 4/100
200/200 [==============================] - 2s 9ms/step - loss: 25406.6191 - precision: 0.1590 - acc: 0.8320 - val_loss: 14772.8779 - val_precision: 0.0000e+00 - val_acc: 0.6979
Epoch 5/100
200/200 [==============================] - 2s 10ms/step - loss: 37452.4844 - precision: 0.2795 - acc: 0.8562 - val_loss: 17342.8047 - val_precision: 0.0000e+00 - val_acc: 0.6979
Epoch 6/100
200/200 [==============================] - 2s 10ms

## 预测概率

In [9]:
print("X_test.target.shape:", X_test.target.shape)
# X = X_test.target.reshape(X_test.target.shape[0], -1)
# print(X.shape)
y_pred = model.predict(X_test.target).transpose()
print("Predicted:", y_pred, y_pred.shape)
print("True:      ", y_test.target, y_test.target.shape)

X_test.target.shape: (17758, 6048)
Predicted: [[0.104 0.104 0.104 0.104 0.104 ... 0.104 0.104 0.104 0.104 0.104]] (1, 17758)
True:       [0.000 0.000 0.000 0.000 0.000 ... 0.000 1.000 0.000 0.000 1.000] (17758,)


## 查看Tensorboard

In [8]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
